In [4]:
import pygame
import sys
import random
# ----- Init Pygame ----- #
pygame.init()

# ----- Screen Setup ----- #
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Don't Touch the Dino")

# ----- Clock ----- #
clock = pygame.time.Clock()
FPS = 60

# ----- Colors ----- #
WHITE = (255, 255, 255)
BLUE = (0, 100, 255)
GREEN = (0, 255, 100)
RED = (255, 0, 0)

# ----- Load Background Image ----- #
background = pygame.image.load("background.jpg")
background = pygame.transform.scale(background, (WIDTH, HEIGHT))

# ---- Enemy Image ------ #
enemy_img = pygame.image.load("enemy.png").convert_alpha()
enemy_img = pygame.transform.scale(enemy_img, (50, 70))

# ----- Player Image ----- #
player_img = pygame.image.load("run.png").convert_alpha()
player_img = pygame.transform.scale(player_img, (70, 70))

# ----- Load Flag Image ----- #
flag_img = pygame.image.load("flag-removebg-preview.png")
flag_img = pygame.transform.scale(flag_img, (70, 100))

# ----- Load Sounds ----- #
pygame.mixer.music.load("Stunted Adventure.mp3")
pygame.mixer.music.play(-1)  # Loop forever

jump_sound = pygame.mixer.Sound("Preview.ogg")
win_sound = pygame.mixer.Sound("win.wav")
lose_sound = pygame.mixer.Sound("lose.flac")

# ----- Player ----- #
player = pygame.Rect(100, 500, 50, 80)
player_vel = 0
jump_strength = -20
on_ground = False
gravity = 0.5

# ----- Platforms ----- #
platforms = [pygame.Rect(0, 440, 800, 100)]

# ----- Enemies ----- #
enemy_timer = 0
enemy_interval = 1500
enemies = [{"rect": pygame.Rect(800, random.choice([150, 270, 360]), 50, 50)}]


# ----- Finish Line ----- #
finish_line = pygame.Rect(740, 350, 70, 50)  # Used for collision
def show_instructions():
    font = pygame.font.SysFont(None, 48)
    small_font = pygame.font.SysFont(None, 36)
    instructions = [
        "How to Play:",
        "Press SPACEBAR to JUMP",
        "Press RIGHT ARROW to MOVE FORWARD",
        "",
        "Click anywhere to continue..."
    ]
    while True:
        screen.blit(background , (0,0))
        y = HEIGHT // 2 - 100
        for line in instructions:
            text = font.render(line, True, (0, 0, 0)) if line == "How to Play:" else small_font.render(line, True, (0, 0, 0))
            screen.blit(text, (WIDTH // 2 - text.get_width() // 2, y))
            y += 50
        pygame.display.flip()
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.MOUSEBUTTONDOWN or event.type == pygame.KEYDOWN:
                return
# --- Set Dificulties ----- #
def setdificulty():
    font = pygame.font.SysFont(None,48)
    Easy_text = font.render("1. Easy" , True , BLUE)
    Medium_text = font.render("2. Medium" , True , GREEN)
    Hard_text = font.render("3. Hard" ,True , RED)

    easy_rect = Easy_text.get_rect(topleft=(WIDTH//2 - 100 , HEIGHT//2 - 40))
    medium_rect = Medium_text.get_rect(topleft=(WIDTH//2 - 100 , HEIGHT//2 + 10))
    hard_rect = Hard_text.get_rect(topleft=(WIDTH//2 - 100 , HEIGHT//2 + 60))

    while True:
        screen.blit(background , (0,0))
        title = font.render("Select Difficulty" , True , (0,0,0))
        screen.blit(title , (WIDTH//2 - 150 , HEIGHT//2 - 120))
        screen.blit(Easy_text , easy_rect.topleft)
        screen.blit(Medium_text , medium_rect.topleft)
        screen.blit(Hard_text , hard_rect.topleft)
        pygame.display.flip()
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_1:
                    return "easy"
                elif event.key == pygame.K_2:
                    return "medium"
                elif event.key == pygame.K_3:
                    return "hard"
            if event.type == pygame.MOUSEBUTTONDOWN:
                mouse_pos = pygame.mouse.get_pos()
                if easy_rect.collidepoint(mouse_pos):
                    return "easy"
                elif medium_rect.collidepoint(mouse_pos):
                    return "medium"
                elif hard_rect.collidepoint(mouse_pos):
                    return "hard"

show_instructions()        
dificulty = setdificulty()

# ------ Set Parameters ------ #
if dificulty == "easy":
    enemy_interval = 1500
    enemy_speed = 5
elif dificulty == "medium":
    enemy_interval = 1000
    enemy_speed = 8
elif dificulty == "hard":
    enemy_interval = 700
    enemy_speed = 12


# ----- Game Loop ----- #
bg_x = 0
bg_speed = 3


running = True
while running:

    bg_x -= bg_speed
    if bg_x <= -WIDTH:
        bg_x = 0
    screen.blit(background , (bg_x,0))
    screen.blit(background , (bg_x + WIDTH,0))
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

    # ----- New Enemies Randomly ----- #


    current_time = pygame.time.get_ticks()
    if current_time - enemy_timer >= enemy_interval:
        if dificulty == "easy":
            y_positions = [random.randint(150, 250), random.randint(270, 350), random.randint(360, 420)]
            new_enemy = {"rect": pygame.Rect(800, random.choice([300, 390, 440]), 50, 50)}
            enemies.append(new_enemy)
        elif dificulty == "medium":
            for _ in range(2): # 2 enemies at a time
                new_enemy = {"rect": pygame.Rect(800, random.choice([300, 390, 440]), 50, 50)}
                enemies.append(new_enemy)
        elif dificulty == "hard":
            for _ in range(3): # 3 enemies at a time
                new_enemy = {"rect": pygame.Rect(800, random.choice([300, 390, 440]), 50, 50)}
                enemies.append(new_enemy)
        enemy_timer = current_time




    # ----- Key Presses ----- #
    keys = pygame.key.get_pressed()
    if keys[pygame.K_LEFT]:
        player.x  -= 5
    if keys[pygame.K_RIGHT]:
        player.x += 5
    if keys[pygame.K_SPACE] and on_ground and player.right < finish_line .left:
        player_vel = jump_strength
        on_ground = False
        jump_sound.play()

    # ----- Gravity ----- #
    player_vel += gravity
    player.y += player_vel

    # ----- Platform Collision ----- #
    on_ground = False
    for plat in platforms:
        if player.colliderect(plat) and player_vel > 0:
            player.bottom = plat.top
            player_vel = 0
            on_ground = True


    # ----- Move Enemies ----- #
    for enemy in enemies[:]:
        enemy["rect"].x -= enemy_speed
        screen.blit(enemy_img, (enemy["rect"].x, enemy["rect"].y))

        if player.colliderect(enemy["rect"]):
            pygame.mixer.music.stop()
            jump_sound.stop()
            lose_sound.play()
            pygame.time.delay(1000)
            font = pygame.font.SysFont(None, 72)
            text = font.render("You Lose!", True, (0, 200, 0))
            screen.blit(text, (WIDTH // 2 - 150, HEIGHT // 2 - 50))
            pygame.display.flip()
            pygame.time.delay(2000)
            pygame.quit()
            sys.exit()

    # ----- Finish Line Collision ----- #
    if player.colliderect(finish_line):
        pygame.mixer.music.stop()
        jump_sound.stop()
        win_sound.play()
        pygame.time.delay(1000)
        font = pygame.font.SysFont(None, 72)
        text = font.render("You Win!", True, (0, 200, 0))
        screen.blit(text, (WIDTH // 2 - 150, HEIGHT // 2 - 50))
        pygame.display.flip()
        pygame.time.delay(2000)
        pygame.quit()
        sys.exit()

    
    # ----- Draw Finish Line ----- #
    screen.blit(flag_img, (finish_line.x, finish_line.y))  
    
    # ----- Draw Player ----- #
    screen.blit(player_img , (player.x , player.y))

    # ----- Update Display & Tick ----- #
    pygame.display.flip()
    clock.tick(FPS)

pygame.quit()
sys.exit()

SystemExit: 